# Testing Notebook

This notebook is to input a model from training and test on new data

In [1]:
# Basic imports
from datetime import datetime
import numpy as np
import glob
import os
import sys
from collections import defaultdict
from sklearn.utils import shuffle

# Data management
import pandas as pd

# Data processing
from ssqueezepy import ssq_cwt

# Model training
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, TimeDistributed
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv1D, MaxPooling1D, BatchNormalization
from tensorflow.keras.layers import LSTM, GRU
from tensorflow.keras.metrics import RootMeanSquaredError
import tensorflow as tf
import matplotlib.pyplot as plt

## Test Visualization

In [ ]:
# Read test EEG data from dat directory
test_file = pd.read_csv('t1_a6_dataset/GO/dk1_test_1673473168.csv', header=None)
test_file

In [ ]:
# Try test file

# Convert to numpy arrays
timestamps, tp9, af7, af8, tp10 = test_file.T.to_numpy()

# Get the frequency of the samples 
start = datetime.strptime(str(timestamps[0]), "%Y-%m-%d %H:%M:%S.%f")
end = datetime.strptime(str(timestamps[-1]), "%Y-%m-%d %H:%M:%S.%f")
duration = (end-start).total_seconds()
fs = timestamps.size/duration

Twtp9, Wtp9, *_ = ssq_cwt(tp9, fs=fs)
fig = plt.figure(figsize=(20, 20))
fig.add_subplot(3, 1, 2)
plt.imshow(np.abs(Twtp9), aspect='auto', cmap='turbo')
fig.add_subplot(3, 1, 1)
plt.imshow(np.abs(Wtp9), aspect='auto', cmap='turbo')
plt.show()

## Importing Data

In [2]:
# Pull all file data
files = defaultdict(list)

# Number of files per category to pull
num_files = len(min((glob.glob('./t1_compatability_dataset/GO/dk*.csv'),
                     glob.glob('./t1_compatability_dataset/STOP/dk*.csv'),
                     glob.glob('./t1_compatability_dataset/REST/dk*.csv')),
                key=len))

for i, file in enumerate(glob.glob('./test_dataset/GO/ridvan2*.csv')):
#     if i == num_files:
#         break
    files["GO"].append(pd.read_csv(file, header=None))
    
for i, file in enumerate(glob.glob('./test_dataset/REST/ridvan2*.csv')):
#     if i == num_files:
#         break
    files["REST"].append(pd.read_csv(file, header=None))
    
for i, file in enumerate(glob.glob('./test_dataset/STOP/ridvan2*.csv')):
#     if i == num_files:
#         break
    files["STOP"].append(pd.read_csv(file, header=None))
    
print(len(files['GO']))
print(len(files['STOP']))
print(len(files['REST']))

4
6
9


## Signal Squeeze Wavelet Transform

In [3]:
def ts_to_ssq(timeseries):
    """
    Converts 4 channel time series data to ssq wavelet transform
    """
    timestamps, tp9, af7, af8, tp10 = timeseries
    
    # Get the frequency of the samples 
    start = datetime.strptime(str(timestamps[0]), "%Y-%m-%d %H:%M:%S.%f")
    end = datetime.strptime(str(timestamps[-1]), "%Y-%m-%d %H:%M:%S.%f")
    duration = (end-start).total_seconds()
    fs = timestamps.size/duration
    
    Twtp9, _, *_ = ssq_cwt(tp9, fs=fs)
    Twtp10, _, *_ = ssq_cwt(tp10, fs=fs)
    Twaf7, _, *_ = ssq_cwt(af7, fs=fs)
    Twaf8, _, *_ = ssq_cwt(af8, fs=fs)
    
    return np.array((Twtp9, Twaf7, Twaf8, Twtp10))

In [4]:
# Compile all files into test data
X = []
t = []

key_dict = {"STOP":(0.0, 0.0, 1.0),
            "GO":(0.0, 1.0, 0.0),
            "REST": (1.0, 0.0, 0.0)}

original_sample_size = 3000
buffer_sample_size = 500
training_sample_size = 25

num_files = sum([len(x) for x in files.values()])
                
for key, data in files.items():
    for recording in data:
        if not num_files%10:
            print("%d files left to convert" % num_files)
        num_files -= 1
        
        # if file is smaller than 3000 samples, don't use it
        if len(recording[0]) < original_sample_size:
            continue

        # Split file into buffer size samples
        time_samples = np.split(recording.head(original_sample_size).T.to_numpy(),
                                     int(original_sample_size/buffer_sample_size), axis=1)
        
        for time_sample in time_samples:
            # Convert to numpy arrays
            ssq_data = ts_to_ssq(time_sample)

            # Take the magnitude of the wavelet transform only
            ssq_data = np.abs(ssq_data)

            # Must transpose data to (frame, height, channel) for training
            ssq_data = ssq_data.transpose(2,1,0)

            # Split sample into many blocks
            wavelet_samples = np.split(ssq_data, int(buffer_sample_size/training_sample_size))

            X.extend(wavelet_samples)
            t.extend([key_dict[key] for i in range(len(wavelet_samples))])
        

10 files left to convert


In [5]:
print(len(X), X[0].shape)
print(len(t), t[0])

2280 (25, 229, 4)
2280 (0.0, 1.0, 0.0)


In [6]:
SEED = 1
X, t = shuffle(np.array(X), np.array(t), random_state=SEED)

# Test Model

In [7]:
model = tf.keras.models.load_model('models/trial1/attempt8_500.h5')

2023-02-02 20:25:25.249914: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2023-02-02 20:25:25.329921: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:21:00.0 name: Tesla K20Xm computeCapability: 3.5
coreClock: 0.732GHz coreCount: 14 deviceMemorySize: 5.57GiB deviceMemoryBandwidth: 232.46GiB/s
2023-02-02 20:25:25.330384: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2023-02-02 20:25:25.333243: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2023-02-02 20:25:25.335163: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2023-02-02 20:25:25.335641: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcurand.so.10
2023-

In [8]:
results = model.evaluate(X, t, batch_size=1)

2023-02-02 20:25:31.362521: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2023-02-02 20:25:31.559959: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7


2280/2280 [==============================] - 42s 18ms/step - loss: 6.3017 - accuracy: 0.4167


In [65]:
tf.keras.backend.clear_session()